In [1]:
import nltk
from pprint import pprint

In [2]:
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()]

In [3]:
stopwords = nltk.corpus.stopwords.words("english")

In [4]:
words =[w for w in words if w.lower() not in stopwords]

In [7]:
fd = nltk.FreqDist(words)

In [8]:
fd.most_common(3)

[('must', 1568), ('people', 1291), ('world', 1128)]

In [9]:
fd.tabulate(3)

  must people  world 
  1568   1291   1128 


In [13]:
text = nltk.Text(nltk.corpus.state_union.words())
text.concordance("america", lines=5)

Displaying 5 of 1079 matches:
 would want us to do . That is what America will do . So much blood has already
ay , the entire world is looking to America for enlightened leadership to peace
beyond any shadow of a doubt , that America will continue the fight for freedom
 to make complete victory certain , America will never become a party to any pl
nly in law and in justice . Here in America , we have labored long and hard to 


In [16]:
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()]
finder = nltk.collocations.TrigramCollocationFinder.from_words(words)

In [20]:
finder.ngram_fd.most_common(2)
finder.ngram_fd.tabulate(2)

  ('the', 'United', 'States') ('the', 'American', 'people') 
                          294                           185 


In [25]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is reall powerful!")
sia.polarity_scores("today was great!")

{'neg': 0.0, 'neu': 0.313, 'pos': 0.687, 'compound': 0.6588}

In [26]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]

In [28]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

shuffle(tweets)
for tweet in tweets[:10]:
    print(">", is_positive(tweet),'::::SPACER::::', tweet)

> False ::::SPACER:::: finished downloading more than 10 movies and one complete tv series from last night. Now… WHERE.DO.I.START. :)))))) #moviemarathon
> True ::::SPACER:::: @fouraroundworld thanks for sharing! Wishing you a wicked weekend :)
> True ::::SPACER:::: RT @abstex: The FT is backing the Tories. On an unrelated note, here's a photo of FT leader writer Jonathan Ford (next to Boris) http//t.c…
> False ::::SPACER:::: The show is called "Ask Nigel Farage", not "Nigel Farage answers what is asked". #AskNigelFarage
> False ::::SPACER:::: Packing is such a nightmare :(
> False ::::SPACER:::: @TheLastLeg #milibrandcuts 'discussing' tory manifesto and Johnny rock-hard?
> True ::::SPACER:::: RT @UKIP: Cheers! Official @UKIP account passes 100k followers #UKIP http//t.co/mdJ2MC95ek
> True ::::SPACER:::: Tomorrow at the #GoldCoast :) http//t.co/yRat44jWAy
> True ::::SPACER:::: RT @davidtorrance: Minority government will allow Ed Miliband to call Nicola Sturgeon’s bluff | Martin Kettle 

In [30]:
positive_review_ids = nltk.corpus.movie_reviews.fileids(categories=["pos"])
negative_review_ids = nltk.corpus.movie_reviews.fileids(categories=["neg"])
all_review_ids = positive_review_ids + negative_review_ids

In [39]:
from statistics import mean
def is_positive(review_id: str) -> bool:
    """True if the average of all sentance compound scores is positive."""
    text = nltk.corpus.movie_reviews.raw(review_id)
    scores = [
        sia.polarity_scores(sentance)["compound"]
        for sentance in nltk.sent_tokenize(text)
    ]
    return mean(scores) > 0

In [42]:
shuffle(all_review_ids)
correct = 0
for review_id in all_review_ids:
    if is_positive(review_id):
        if review_id in positive_review_ids:
            correct += 1
    else: 
        if review_id in negative_review_ids:
            correct += 1

print(F"{correct / len(all_review_ids):.2%} correct")

64.00% correct
